In [7]:
import pandas as pd
from src.data_loader import *
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import Audio

In [8]:
data_path = Path.cwd()/'..'/'data'
df = pd.read_csv(data_path/'label.csv')
class_columns = [col for col in df.columns if col not in ['relative_path', 'data_origin']]

In [9]:
ds = SoundDS(df, data_path, class_columns)
print(len(ds))

spectrogram, class_labels = ds[0]

26863


In [10]:
df.head()

,no voice,singer,duet,plucking,hard rock,world,bongos,harpsichord,sitar,chorus,...,rap,metal,hiphop,quick,water,baroque,fiddle,english,filepath,data_origin
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,mtat/f/american_bach_soloists-j_s__bach_solo_c...,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,mtat/f/american_bach_soloists-j_s__bach_solo_c...,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,mtat/f/american_bach_soloists-j_s__bach_solo_c...,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,mtat/f/american_bach_soloists-j_s__bach_solo_c...,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,mtat/f/american_bach_soloists-j_s__bach_solo_c...,1


In [14]:
audio_path = df['filepath'][0]
print(data_path)

audio_file = data_path/audio_path
aud = AudioUtil.open(audio_file)
num_aud_channel = AudioUtil.get_audio_channels(audio_file)
print(num_aud_channel)

/Users/ab/Projects/Tagging-Music-Sequences/notebooks/../data
1


In [6]:
Audio(audio_file)

In [7]:
df_subset = df[df['data_origin'] == 0]
df_subset.shape

(1000, 171)

# EDA through entire dataset

In [8]:
# Initialize an empty list to store the data
audio_data = []
error_files = []  # List to store paths of problematic files

# Iterate over the filepaths in the subset DataFrame
for value in df['filepath']:
    try:
        # Construct the full file path for the audio file
        full_path = data_path / value
        # Use your AudioUtil class to open the audio file
        audio, sr = AudioUtil.open(full_path)
        num_channels = AudioUtil.get_audio_channels(full_path)
        audio_length = audio.shape[1] / sr

        # Create a dictionary of the data you want to store
        audio_info = {
            'filepath': full_path,
            'audio_channels': num_channels, 
            'sample': audio,
            'sampling_rate': sr,
            'audio_length': audio_length,
            #TODO more features
        }

        # Append the dictionary to your list
        audio_data.append(audio_info)
        
    except Exception as e: 
        print(f"Failed to process {value}: {e}")
        error_files.append({'filepath': value, 'error': str(e)})

# Convert the list of dictionaries to a new DataFrame
audio_df = pd.DataFrame(audio_data)
error_df = pd.DataFrame(error_files)

Failed to process mtat/6/norine_braun-now_and_zen-08-gently-117-146.mp3: Failed to open the input "/Users/ab/Projects/Tagging-Music-Sequences/data/mtat/6/norine_braun-now_and_zen-08-gently-117-146.mp3" (Invalid argument).
Failed to process mtat/8/jacob_heringman-josquin_des_prez_lute_settings-19-gintzler__pater_noster-204-233.mp3: Failed to open the input "/Users/ab/Projects/Tagging-Music-Sequences/data/mtat/8/jacob_heringman-josquin_des_prez_lute_settings-19-gintzler__pater_noster-204-233.mp3" (Invalid argument).
Failed to process mtat/9/american_baroque-dances_and_suites_of_rameau_and_couperin-25-le_petit_rien_xiveme_ordre_couperin-88-117.mp3: Failed to open the input "/Users/ab/Projects/Tagging-Music-Sequences/data/mtat/9/american_baroque-dances_and_suites_of_rameau_and_couperin-25-le_petit_rien_xiveme_ordre_couperin-88-117.mp3" (Invalid argument).
Failed to process gtzan/genres_original/jazz/jazz.00054.wav: Failed to open the input "/Users/ab/Projects/Tagging-Music-Sequences/data/g

In [9]:
audio_df

,filepath,audio channels,sampling_rate,audio_length
0,/Users/ab/Projects/Tagging-Music-Sequences/not...,1,16000,29.124000
1,/Users/ab/Projects/Tagging-Music-Sequences/not...,1,16000,29.124000
2,/Users/ab/Projects/Tagging-Music-Sequences/not...,1,16000,29.124000
3,/Users/ab/Projects/Tagging-Music-Sequences/not...,1,16000,29.124000
4,/Users/ab/Projects/Tagging-Music-Sequences/not...,1,16000,29.124000
...,...,...,...,...
26854,/Users/ab/Projects/Tagging-Music-Sequences/not...,1,22050,30.013333
26855,/Users/ab/Projects/Tagging-Music-Sequences/not...,1,22050,30.013333
26856,/Users/ab/Projects/Tagging-Music-Sequences/not...,1,22050,30.013333
26857,/Users/ab/Projects/Tagging-Music-Sequences/not...,1,22050,30.013333


In [11]:
columns = ['audio_channels', 'sampling_rate', 'audio_length'] 

# Print unique values for each specified column
for column in columns:
    print(f"Unique values in '{column}': {audio_df[column].unique()}")
    


Unique values in 'audio channels': [1]
Unique values in 'sampling_rate': [16000 22050]
Unique values in 'audio_length': [29.124      30.01333333 30.37097506 30.00798186 30.17142857 30.01179138
 30.09160998 29.99582766 29.99292517 30.39092971 30.48897959 30.07165533
 30.08163265 30.24126984 30.33106576 29.98185941 30.1015873  30.12154195
 30.21133787 30.30113379 30.29115646 30.16145125 30.00018141 30.13151927
 30.35102041 29.93197279 30.28117914 30.64888889 30.20136054 30.48072562
 30.02176871 30.27120181 30.         30.40090703 30.36099773]


### Observations

- varying sampling rates in the datasets:
    - mtat uses 16000
    - gtzan uses 22050
- varying song lengths across tracks
- ...